In [ ]:
from tensorflow.python.client import device_lib 
import tensorflow as tf

print(device_lib.list_local_devices())
print(tf.__version__)

In [51]:

from os import listdir
import os
import scipy.io
from scipy.ndimage import rotate
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from PIL import Image
import random
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
import random
    
    
# --------------------------------------------------LOAD DATA------------------------------------------------------------------


def loadData(dataset_folder, 
             test_folder, 
             columns, 
             batch_size, 
             input_shape,
            ):
    
    # step 1
    train_files = [f"data/{dataset_folder}/{x}" for x in os.listdir(f"data/{dataset_folder}")]
    test_files = [f"data/{test_folder}/{x}" for x in os.listdir(f"data/{test_folder}")]

    train_df = pd.read_parquet(f"data/{dataset_folder}_dataset.parquet.gzip")
    test_df = pd.read_parquet(f"data/{test_folder}_dataset.parquet.gzip")
    
    train_df.index = f"data/{dataset_folder}/" + train_df["file"]
    test_df.index =  f"data/{test_folder}/" + test_df["file"]
    
    train_df.drop(columns=['file'], inplace=True)
    test_df.drop(columns=['file'], inplace=True)

    train_y = train_df.loc[train_files][columns].to_numpy()
    test_y =test_df.loc[test_files][columns].to_numpy()
    
        
    def augment(im):
        
        pad_size = random.randint(0, 24)
        im = tf.image.resize(im, [224+pad_size,224+pad_size])
        # Flip
        im = tf.image.random_flip_left_right(
            im, seed=None
        )
        # Saturation
        im = tf.image.random_saturation(
            im, 0, 3
        )
        # Contrast
        im = tf.image.random_contrast(
            im, 0.5, 1
        )
        # Brightness
        im = tf.image.random_brightness(
            im, 0.4
        )
        #Crop

        # pad_size = random.randint(0, 24)
        size = 224

        padded = np.ones((size+24, size+24, 3))
        x_shift = random.randint(0, 24 - pad_size)
        y_shift = random.randint(0, 24 - pad_size)
        xMax = size + pad_size + x_shift
        yMax = size + pad_size + y_shift
        padded[y_shift : yMax, x_shift : xMax, :] = im

        x_shift = random.randint(0, pad_size)
        y_shift = random.randint(0, pad_size)
        im = padded[y_shift:size+y_shift, x_shift:size+x_shift, :]

        return im

    
    
    def im_file_to_tensor(file, label):
        
        def _transform(im, label):
            im = augment(im)
            im = tf.cast(im, tf.float32) / 255.0
            label = tf.cast(label, tf.float32)
            return im, label
        
        def _im_file_to_tensor(file, label):
            path = file.numpy().decode()
            im = tf.image.decode_png(tf.io.read_file(path), channels=3)
            im, label = _transform(im, label)
            
            return im, label
        
        file, label =  tf.py_function(_im_file_to_tensor, 
                              inp=(file, label), 
                              Tout=(tf.float32, tf.float32))

        return ({"image" : file}, label)
    
        
    train = tf.data.Dataset.from_tensor_slices((train_files, train_y))
    test = tf.data.Dataset.from_tensor_slices((test_files, test_y))

    train_ds = train.map(im_file_to_tensor, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds  = test.map(im_file_to_tensor, num_parallel_calls=tf.data.AUTOTUNE)
    
    train_ds = train_ds.batch(24, drop_remainder=False).prefetch(tf.data.AUTOTUNE)
    test_ds  = test_ds.batch(24, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

        
    return train_ds, test_ds


# --------------------------------------------------LOAD MODEL-------------------------------------------------------------------

from keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Concatenate, BatchNormalization, Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import keras
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    TensorBoard,
)
import tensorflow as tf
from keras import backend as K

def loadModel(name, loss, input_shape, output_shape):

    resnet = ResNet50V2(include_top=False)
    
    inputsA = keras.Input(shape=input_shape, name="image")
    xA = resnet(inputsA, training=True)
    xA = keras.layers.GlobalAveragePooling2D()(xA)
    output = Dense(output_shape)(xA)
    model = Model(inputs=inputsA, outputs=output)
    return model
    

# --------------------------------------------------TRAIN-------------------------------------------------------------------

policy = tf.keras.mixed_precision.Policy('mixed_float16') # mixed_float16
tf.keras.mixed_precision.set_global_policy(policy)


def trainModel(model, train_dataset, test_dataset, model_name, version, epochs, patience, loss):
    
    early_stopping_callback = EarlyStopping(
        monitor="val_loss", mode="min", verbose=1, patience=patience
    )

    tensorboard_callback = TensorBoard(log_dir=f"logs/{model_name}_{version}")
    
    opt=keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss=loss,
        optimizer="adam",
        metrics=[
            tf.keras.metrics.MeanSquaredError(),
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )

    model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        callbacks=[
            early_stopping_callback,
            tensorboard_callback
        ],
    )
        
    model.save(f"model/{model_name}/{version}")

In [52]:

DATASET = 'train_aws'
TEST_DATASET = 'testA_aws'

FULL_DATASET = True
MODEL_ONLY = False
TRAINING = True
SHOW_RESULTS = False

PREDICTION_NAME =  ['waist']
MODEL_NAME = 'resnet50'
MODEL_VERSION = 'v1'
INPUT_SHAPE = (224, 224, 3)
OUTPUT_SHAPE = len(PREDICTION_NAME)
EPOCHS = 5000
PATIENCE = 30
BATCH_SIZE = 24
LOSS = 'mse'


In [ ]:


keras.backend.clear_session()


print('Loading Data')
train_dataset, test_dataset = loadData(
    DATASET, TEST_DATASET, PREDICTION_NAME, BATCH_SIZE, INPUT_SHAPE
)
print('Loaded')

model = loadModel(MODEL_NAME, LOSS, INPUT_SHAPE, OUTPUT_SHAPE)

trainModel(model, train_dataset, test_dataset, MODEL_NAME, MODEL_VERSION, EPOCHS, PATIENCE, LOSS)
